<a href="https://colab.research.google.com/github/RaiAnant/MangaChroma/blob/master/fasterai/loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

Fri Aug 16 11:51:44 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Imports

In [0]:
from fastai import *
from fastai.core import *
from fastai.torch_core import *
from fastai.callbacks  import hook_outputs
import torchvision.models as models

## Loss classes

In [0]:
# m = models.vgg16_bn(True)

In [0]:
# ??models.vgg16_bn

In [0]:
# ??models.vgg

In [0]:
# ??models.vgg.make_layers

In [0]:
# m = models.vgg16_bn(True).features.cuda().eval()
# blocks = [i-1 for i,o in enumerate(children(m)) if isinstance(o,nn.MaxPool2d)]
# l = [m[i] for i in blocks[2:5]]
# l

[ReLU(inplace), ReLU(inplace), ReLU(inplace)]

In [0]:
class FeatureLoss(nn.Module):
  """Feature loss required for pretraining Generator"""
  def __init__(self, layer_wts=[20,70,10]):
    super().__init__()
    # Feature extractor
    # .features gets the layers of the model with pretrained weights 
    self.m_feat = models.vgg16_bn(True).features.cuda().eval()
    requires_grad(self.m_feat, False)
    # Selecting the activation layers just before downsampling occurs
    blocks = [i-1 for i,o in enumerate(children(self.m_feat)) if isinstance(o,nn.MaxPool2d)]
    layers_ids = blocks[2:5]
    self.loss_features = [self.m_feat[i] for i in layer_ids]
    # Takes the outputs of these specific layers which 
    self.hooks = hook_outputs(self.loss_features, detach=False)
    self.wgts = layer_wgts
    self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))]
    self.base_loss = F.l1_loss
  
  def _make_features(self, x, clone=False):
    self.m_feat(x)
    # .clone() copies the tensor while still keeping the copy attached to the graph
    return [(o.clone() if clone else o) for o in self.hooks.stored]

  def forward(self, input, target):
    out_feat = self._make_features(target, clone=True) # Extract features for the target image (Colored)
    in_feat = self._make_features(input) # Extract features for the tfmed image
    self.feat_losses = [self.base_loss(input,target)]
    self.feat_losses += [self.base_loss(f_in, f_out)*w
                         for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
    
    self.metrics = dict(zip(self.metric_names, self.feat_losses))
    return sum(self.feat_losses)

  def __del__(self): self.hooks.remove()